<a href="https://colab.research.google.com/github/BaronVonBussin/Stuff/blob/main/lstm_20241218.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install yfinance pandas numpy matplotlib seaborn tensorflow plotly

In [4]:
pip install keras

In [8]:
!pip uninstall tensorflow tensorflow-gpu
!pip install tensorflow

Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.17.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.17.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 86.3 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the 

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly.express as px

# Fetch SPY data
def get_spy_data(start_date, end_date):
    df = yf.download('SPY', start=start_date, end=end_date)
    df = df[df.index.dayofweek < 5]  # Remove weekends
    return df

# Prepare data for LSTM
def prepare_lstm_data(data, lookback):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)

    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    return X_train, X_test, y_train, y_test, scaler

# Create and train LSTM model
def create_lstm_model(lookback):
    model = Sequential([
        LSTM(50, input_shape=(lookback, 1), return_sequences=False),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Create visualization with prediction analysis
def create_analysis_plot(dates, actual_values, predictions, mse_values):
    # Create figure with secondary y-axis
    fig = go.Figure()

    # Add actual prices
    fig.add_trace(
        go.Scatter(x=dates, y=actual_values,
                  name="Actual Price",
                  line=dict(color='blue'),
                  opacity=0.7)
    )

    # Add predictions
    fig.add_trace(
        go.Scatter(x=dates, y=predictions,
                  name="LSTM Prediction",
                  line=dict(color='red'),
                  opacity=0.7)
    )

    # Add MSE as area plot on secondary axis
    fig.add_trace(
        go.Scatter(x=dates, y=mse_values,
                  name="Prediction Error",
                  fill='tozeroy',
                  yaxis='y2',
                  line=dict(color='rgba(0,255,0,0.2)'),
                  opacity=0.3)
    )

    # Update layout
    fig.update_layout(
        title="SPY Price Prediction Analysis with LSTM (50-day lookback)",
        xaxis_title="Date",
        yaxis_title="Price ($)",
        yaxis2=dict(
            title="Prediction Error",
            overlaying="y",
            side="right",
            showgrid=False
        ),
        hovermode='x unified',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    return fig

def main():
    # Parameters
    start_date = '2014-01-01'
    end_date = '2017-12-31'
    lookback = 50

    # Get data
    df = get_spy_data(start_date, end_date)

    # Prepare data
    X_train, X_test, y_train, y_test, scaler = prepare_lstm_data(df, lookback)

    # Create and train model
    model = create_lstm_model(lookback)
    history = model.fit(X_train, y_train,
                       epochs=50,
                       batch_size=32,
                       validation_split=0.1,
                       verbose=0)

    # Generate predictions
    test_predictions = model.predict(X_test)

    # Transform predictions back to original scale
    test_predictions = scaler.inverse_transform(test_predictions)
    actual_values = scaler.inverse_transform(y_test)

    # Calculate MSE for each prediction
    mse_values = np.square(test_predictions - actual_values)

    # Get dates for test period
    test_dates = df.index[-(len(test_predictions)):]

    # Create and show visualization
    fig = create_analysis_plot(
        test_dates,
        actual_values.flatten(),
        test_predictions.flatten(),
        mse_values.flatten()
    )
    fig.show()

    # Print performance metrics
    print("\nLSTM Model Analysis Results:")
    print("-" * 50)
    print(f"Mean Absolute Error: ${np.mean(np.abs(test_predictions - actual_values)):.2f}")
    print(f"Root Mean Square Error: ${np.sqrt(np.mean(np.square(test_predictions - actual_values))):.2f}")
    print(f"Mean Prediction Error %: {np.mean(np.abs(test_predictions - actual_values) / actual_values) * 100:.2f}%")
    print("-" * 50)

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step  



LSTM Model Analysis Results:
--------------------------------------------------
Mean Absolute Error: $1.97
Root Mean Square Error: $2.25
Mean Prediction Error %: 0.79%
--------------------------------------------------


In [2]:
# Fetch SPY data
def get_spy_data(start_date, end_date):
    df = yf.download('SPY', start=start_date, end=end_date)
    df = df[df.index.dayofweek < 5]  # Remove weekends
    return df

# Prepare data for LSTM
def prepare_lstm_data(data, lookback):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)

    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    return X_train, X_test, y_train, y_test, scaler

# Create and train LSTM model
def create_lstm_model(lookback):
    model = Sequential([
        LSTM(50, input_shape=(lookback, 1), return_sequences=False),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Create visualization with prediction analysis
def create_analysis_plot(dates, actual_values, predictions, mse_values):
    # Create figure with secondary y-axis
    fig = go.Figure()

    # Add actual prices
    fig.add_trace(
        go.Scatter(x=dates, y=actual_values,
                  name="Actual Price",
                  line=dict(color='blue'),
                  opacity=0.7)
    )

    # Add predictions
    fig.add_trace(
        go.Scatter(x=dates, y=predictions,
                  name="LSTM Prediction",
                  line=dict(color='red'),
                  opacity=0.7)
    )

    # Add MSE as area plot on secondary axis
    fig.add_trace(
        go.Scatter(x=dates, y=mse_values,
                  name="Prediction Error",
                  fill='tozeroy',
                  yaxis='y2',
                  line=dict(color='rgba(0,255,0,0.2)'),
                  opacity=0.3)
    )

    # Update layout
    fig.update_layout(
        title="SPY Price Prediction Analysis with LSTM (200-day lookback)",
        xaxis_title="Date",
        yaxis_title="Price ($)",
        yaxis2=dict(
            title="Prediction Error",
            overlaying="y",
            side="right",
            showgrid=False
        ),
        hovermode='x unified',
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    return fig

def main():
    # Parameters
    start_date = '2014-01-01'
    end_date = '2017-12-31'
    lookback = 200

    # Get data
    df = get_spy_data(start_date, end_date)

    # Prepare data
    X_train, X_test, y_train, y_test, scaler = prepare_lstm_data(df, lookback)

    # Create and train model
    model = create_lstm_model(lookback)
    history = model.fit(X_train, y_train,
                       epochs=50,
                       batch_size=32,
                       validation_split=0.1,
                       verbose=0)

    # Generate predictions
    test_predictions = model.predict(X_test)

    # Transform predictions back to original scale
    test_predictions = scaler.inverse_transform(test_predictions)
    actual_values = scaler.inverse_transform(y_test)

    # Calculate MSE for each prediction
    mse_values = np.square(test_predictions - actual_values)

    # Get dates for test period
    test_dates = df.index[-(len(test_predictions)):]

    # Create and show visualization
    fig = create_analysis_plot(
        test_dates,
        actual_values.flatten(),
        test_predictions.flatten(),
        mse_values.flatten()
    )
    fig.show()

    # Print performance metrics
    print("\nLSTM Model Analysis Results:")
    print("-" * 50)
    print(f"Mean Absolute Error: ${np.mean(np.abs(test_predictions - actual_values)):.2f}")
    print(f"Root Mean Square Error: ${np.sqrt(np.mean(np.square(test_predictions - actual_values))):.2f}")
    print(f"Mean Prediction Error %: {np.mean(np.abs(test_predictions - actual_values) / actual_values) * 100:.2f}%")
    print("-" * 50)

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step



LSTM Model Analysis Results:
--------------------------------------------------
Mean Absolute Error: $3.44
Root Mean Square Error: $4.08
Mean Prediction Error %: 1.35%
--------------------------------------------------


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import tensorflow as tf

# Fetch SPY data
def get_spy_data(start_date, end_date):
    df = yf.download('SPY', start=start_date, end=end_date)
    df = df[df.index.dayofweek < 5]  # Remove weekends
    return df

# Create LSTM model with attention tracking
class AttentionLSTM(tf.keras.Model):
    def __init__(self, lookback):
        super(AttentionLSTM, self).__init__()
        self.lstm = LSTM(50, return_sequences=True)
        self.attention = Dense(1, activation='tanh')
        self.dense = Dense(1)

    def call(self, inputs):
        lstm_out = self.lstm(inputs)
        attention_weights = tf.nn.softmax(self.attention(lstm_out), axis=1)
        context = attention_weights * lstm_out
        context = tf.reduce_sum(context, axis=1)
        output = self.dense(context)
        return output, attention_weights

# Prepare data for LSTM
def prepare_data(data, lookback):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)

    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    return X_train, X_test, y_train, y_test, scaler

def create_visualization(dates, prices, predictions, attention_weights, window_size=50):
    # Create figure with secondary y-axis
    fig = make_subplots(rows=2, cols=1,
                       shared_xaxes=True,
                       vertical_spacing=0.05,
                       row_heights=[0.7, 0.3])

    # Add price and prediction traces
    fig.add_trace(
        go.Scatter(x=dates, y=prices, name="Actual Price",
                  line=dict(color='blue')),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=dates, y=predictions, name="LSTM Prediction",
                  line=dict(color='red')),
        row=1, col=1
    )

    # Add attention heatmap
    attention_weights = attention_weights.reshape(-1, window_size)

    fig.add_trace(
        go.Heatmap(
            z=attention_weights,
            x=np.arange(window_size),
            y=dates,
            colorscale='Viridis',
            name='Pattern Importance',
            showscale=True,
            colorbar=dict(title='Pattern Importance')
        ),
        row=2, col=1
    )

    # Update layout
    fig.update_layout(
        title="SPY Price Prediction with Pattern Importance Analysis",
        xaxis_title="Date",
        yaxis_title="Price ($)",
        yaxis2_title="Lookback Days",
        height=800,
        showlegend=True,
        hovermode='x unified'
    )

    return fig

def main():
    # Parameters
    start_date = '2014-01-01'
    end_date = '2017-12-31'
    lookback = 50

    # Get data
    df = get_spy_data(start_date, end_date)

    # Prepare data
    X_train, X_test, y_train, y_test, scaler = prepare_data(df, lookback)

    # Create and train model
    model = AttentionLSTM(lookback)
    model.compile(optimizer='adam', loss='mse')

    # Reshape data for attention model
    X_train_reshaped = X_train.reshape(-1, lookback, 1)
    y_train_reshaped = y_train.reshape(-1, 1)

    # Train model
    model.fit(X_train_reshaped, y_train_reshaped,
             epochs=50, batch_size=32, verbose=0)

    # Generate predictions and get attention weights
    X_test_reshaped = X_test.reshape(-1, lookback, 1)
    predictions, attention_weights = model(X_test_reshaped)

    # Transform predictions back to original scale
    predictions = scaler.inverse_transform(predictions.numpy())
    actual_values = scaler.inverse_transform(y_test)

    # Get dates for test period
    test_dates = df.index[-(len(predictions)):]

    # Create visualization
    fig = create_visualization(
        test_dates,
        actual_values.flatten(),
        predictions.flatten(),
        attention_weights.numpy(),
        lookback
    )
    fig.show()

    # Print interpretation guide
    print("\nHow to Interpret the Visualization:")
    print("-" * 50)
    print("1. Top Panel:")
    print("   - Blue line: Actual SPY price")
    print("   - Red line: LSTM predictions")
    print("\n2. Bottom Panel (Heatmap):")
    print("   - Each row represents one prediction")
    print("   - Colors show which past days were most important")
    print("   - Brighter colors = More important for prediction")
    print("   - X-axis shows lookback days (0 = most recent)")
    print("-" * 50)

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed



How to Interpret the Visualization:
--------------------------------------------------
1. Top Panel:
   - Blue line: Actual SPY price
   - Red line: LSTM predictions

2. Bottom Panel (Heatmap):
   - Each row represents one prediction
   - Colors show which past days were most important
   - Brighter colors = More important for prediction
   - X-axis shows lookback days (0 = most recent)
--------------------------------------------------


In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import json
from pathlib import Path
import tensorflow as tf

class RegimeLSTM:
    def __init__(self, lookback=50):
        self.lookback = lookback
        self.scaler = MinMaxScaler()
        self.models = {}
        self.regime_metadata = {}

    def create_model(self):
        """Create model using functional API"""
        inputs = Input(shape=(self.lookback, 1))
        lstm_out = LSTM(50)(inputs)
        outputs = Dense(1)(lstm_out)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='mse')
        return model

    def prepare_data(self, prices, for_prediction=False):
        """Prepare data with validation checks"""
        if for_prediction:
            if len(prices) != self.lookback:
                raise ValueError(f"For prediction, need exactly {self.lookback} price points, got {len(prices)}")
        else:
            if len(prices) < self.lookback + 1:
                raise ValueError(f"For training, need at least {self.lookback + 1} price points, got {len(prices)}")

        scaled_data = self.scaler.fit_transform(prices.reshape(-1, 1))

        if for_prediction:
            X = np.array([scaled_data])
            return X, None
        else:
            X, y = [], []
            for i in range(self.lookback, len(scaled_data)):
                X.append(scaled_data[i-self.lookback:i])
                y.append(scaled_data[i])
            return np.array(X), np.array(y)

    def train_regime(self, prices, regime_name, regime_characteristics):
        """Train and save model with error handling"""
        try:
            X, y = self.prepare_data(prices, for_prediction=False)

            # Calculate appropriate batch size
            batch_size = min(32, len(X))

            # Create and train model
            model = self.create_model()

            print(f"\nTraining {regime_name} model...")
            print(f"Training data shape: {X.shape}")

            history = model.fit(
                X, y,
                epochs=50,
                batch_size=batch_size,
                validation_split=0.2,
                verbose=1
            )

            # Create save directory
            save_dir = Path("regime_models")
            save_dir.mkdir(exist_ok=True)

            # Save model in .keras format
            model_path = save_dir / f"{regime_name}_model.keras"
            metadata_path = save_dir / f"{regime_name}_metadata.json"

            model.save(model_path)

            # Save metadata
            metadata = {
                "regime_name": regime_name,
                "characteristics": regime_characteristics,
                "training_loss": float(history.history['loss'][-1]),
                "data_points": len(prices),
                "lookback": self.lookback,
                "scaler_params": {
                    "scale_": self.scaler.scale_.tolist(),
                    "min_": self.scaler.min_.tolist(),
                }
            }

            with open(metadata_path, 'w') as f:
                json.dump(metadata, f, indent=4)

            self.models[regime_name] = model
            self.regime_metadata[regime_name] = metadata

            print(f"Final loss: {history.history['loss'][-1]:.6f}")
            print(f"Training data points: {len(prices)}")

            return model, metadata

        except Exception as e:
            print(f"Error training regime {regime_name}: {str(e)}")
            raise

    def predict_with_regime(self, prices, regime_name):
        """Make predictions with error handling"""
        try:
            if regime_name not in self.models:
                self.load_regime_model(regime_name)

            if len(prices) != self.lookback:
                prices = prices[-self.lookback:]  # Take last lookback prices
                print(f"Adjusted input to last {self.lookback} prices")

            # Prepare data specifically for prediction
            X, _ = self.prepare_data(prices, for_prediction=True)

            # Make prediction
            prediction = self.models[regime_name].predict(X, verbose=0)

            # Inverse transform
            prediction = self.scaler.inverse_transform(prediction)

            return float(prediction[0][0])

        except Exception as e:
            print(f"Error predicting with regime {regime_name}: {str(e)}")
            raise

def main():
    try:
        print("Fetching SPY data...")
        df = yf.download('SPY', start='2014-01-01', end='2017-12-31')
        if df.empty:
            raise ValueError("No data downloaded")

        print(f"Downloaded {len(df)} days of data")

        # Create regime classifier
        regime_lstm = RegimeLSTM(lookback=50)

        # Define example regimes
        regimes = {
            "high_vol_uptrend": {
                "volatility": "high",
                "trend": "upward",
                "volume": "above_average"
            },
            "low_vol_sideways": {
                "volatility": "low",
                "trend": "sideways",
                "volume": "below_average"
            }
        }

        print("\nTraining regime models...")

        # Train models
        high_vol_period = df['Close'].values[:500]
        model_high_vol, metadata_high_vol = regime_lstm.train_regime(
            high_vol_period,
            "high_vol_uptrend",
            regimes["high_vol_uptrend"]
        )

        low_vol_period = df['Close'].values[500:1000]
        model_low_vol, metadata_low_vol = regime_lstm.train_regime(
            low_vol_period,
            "low_vol_sideways",
            regimes["low_vol_sideways"]
        )

        # Get exactly lookback days for prediction
        recent_prices = df['Close'].values[-regime_lstm.lookback:]
        print(f"\nMaking predictions using last {len(recent_prices)} price points")

        prediction_high_vol = regime_lstm.predict_with_regime(
            recent_prices,
            "high_vol_uptrend"
        )

        prediction_low_vol = regime_lstm.predict_with_regime(
            recent_prices,
            "low_vol_sideways"
        )

        # Print results
        print("\nRegime-Based LSTM Analysis")
        print("-" * 50)
        current_price = float(df['Close'].values[-1])
        print(f"Current Price: ${current_price:.2f}")
        print(f"High Volatility Regime Prediction: ${prediction_high_vol:.2f} ({((prediction_high_vol/current_price) - 1)*100:.2f}% change)")
        print(f"Low Volatility Regime Prediction: ${prediction_low_vol:.2f} ({((prediction_low_vol/current_price) - 1)*100:.2f}% change)")

        # Print regime characteristics
        print("\nStored Regime Characteristics:")
        for regime_name, metadata in regime_lstm.regime_metadata.items():
            print(f"\n{regime_name}:")
            for key, value in metadata['characteristics'].items():
                print(f"  {key}: {value}")

    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed

Fetching SPY data...
Downloaded 1007 days of data

Training regime models...

Training high_vol_uptrend model...
Training data shape: (450, 50, 1)
Epoch 1/50


12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.2638 - val_loss: 0.0580
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0339 - val_loss: 0.0177
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0125 - val_loss: 0.0149
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0066 - val_loss: 0.0171
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0054 - val_loss: 0.0145
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0056 - val_loss: 0.0158
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0050 - val_loss: 0.0148
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0050 - val_loss: 0.0150
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0045 - val_loss: 0.0150
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0039 - val_loss: 0.0157
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0049 - val_loss: 0.0149
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0043 - val_l


Regime-Based LSTM Analysis
--------------------------------------------------
Current Price: $266.86
High Volatility Regime Prediction: $267.29 (0.16% change)
Low Volatility Regime Prediction: $267.32 (0.17% change)

Stored Regime Characteristics:

high_vol_uptrend:
  volatility: high
  trend: upward
  volume: above_average

low_vol_sideways:
  volatility: low
  trend: sideways
  volume: below_average


<ipython-input-9-d44dff01ab1c>:193: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import json
from pathlib import Path
import tensorflow as tf

# Define prediction function at module level
@tf.function(reduce_retracing=True, jit_compile=True)
def make_prediction(model, data):
    return model(data, training=False)

class RegimeLSTM:
    def __init__(self, lookback=50):
        self.lookback = lookback
        self.scaler = MinMaxScaler()
        self.models = {}
        self.regime_metadata = {}

    def create_model(self):
        """Create model using functional API"""
        inputs = Input(shape=(self.lookback, 1))
        lstm_out = LSTM(50)(inputs)
        outputs = Dense(1)(lstm_out)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='mse')
        return model

    def prepare_data(self, prices, for_prediction=False):
        """Prepare data with validation and convert to tensor"""
        if for_prediction:
            if len(prices) != self.lookback:
                raise ValueError(f"For prediction, need exactly {self.lookback} price points, got {len(prices)}")
            scaled_data = self.scaler.fit_transform(prices.reshape(-1, 1))
            # Convert to tensor with explicit shape
            return tf.convert_to_tensor(scaled_data.reshape(1, self.lookback, 1), dtype=tf.float32), None
        else:
            if len(prices) < self.lookback + 1:
                raise ValueError(f"For training, need at least {self.lookback + 1} price points, got {len(prices)}")

            scaled_data = self.scaler.fit_transform(prices.reshape(-1, 1))
            X, y = [], []
            for i in range(self.lookback, len(scaled_data)):
                X.append(scaled_data[i-self.lookback:i])
                y.append(scaled_data[i])
            # Convert to tensors with explicit shapes
            X_tensor = tf.convert_to_tensor(np.array(X), dtype=tf.float32)
            y_tensor = tf.convert_to_tensor(np.array(y), dtype=tf.float32)
            return X_tensor, y_tensor

    def train_regime(self, prices, regime_name, regime_characteristics):
        """Train and save model with error handling"""
        try:
            X, y = self.prepare_data(prices, for_prediction=False)

            # Calculate appropriate batch size
            batch_size = min(32, len(X))

            # Create and train model
            model = self.create_model()

            print(f"\nTraining {regime_name} model...")
            print(f"Training data shape: {X.shape}")

            history = model.fit(
                X, y,
                epochs=50,
                batch_size=batch_size,
                validation_split=0.2,
                verbose=1
            )

            # Create save directory
            save_dir = Path("regime_models")
            save_dir.mkdir(exist_ok=True)

            # Save model in .keras format
            model_path = save_dir / f"{regime_name}_model.keras"
            metadata_path = save_dir / f"{regime_name}_metadata.json"

            model.save(model_path)

            # Save metadata
            metadata = {
                "regime_name": regime_name,
                "characteristics": regime_characteristics,
                "training_loss": float(history.history['loss'][-1]),
                "data_points": len(prices),
                "lookback": self.lookback,
                "scaler_params": {
                    "scale_": self.scaler.scale_.tolist(),
                    "min_": self.scaler.min_.tolist(),
                }
            }

            with open(metadata_path, 'w') as f:
                json.dump(metadata, f, indent=4)

            self.models[regime_name] = model
            self.regime_metadata[regime_name] = metadata

            print(f"Final loss: {history.history['loss'][-1]:.6f}")
            print(f"Training data points: {len(prices)}")

            return model, metadata

        except Exception as e:
            print(f"Error training regime {regime_name}: {str(e)}")
            raise

    def predict_with_regime(self, prices, regime_name):
        """Make predictions with optimized TensorFlow handling"""
        try:
            if regime_name not in self.models:
                self.load_regime_model(regime_name)

            if len(prices) != self.lookback:
                prices = prices[-self.lookback:]
                print(f"Adjusted input to last {self.lookback} prices")

            # Prepare data as tensor
            X, _ = self.prepare_data(prices, for_prediction=True)

            # Make prediction using global function
            prediction = make_prediction(self.models[regime_name], X)

            # Convert to numpy and inverse transform
            prediction_np = prediction.numpy()
            prediction_orig = self.scaler.inverse_transform(prediction_np)

            return float(prediction_orig[0][0])

        except Exception as e:
            print(f"Error predicting with regime {regime_name}: {str(e)}")
            raise

    def load_regime_model(self, regime_name):
        """Load a pre-trained model for a specific regime"""
        try:
            save_dir = Path("regime_models")
            model_path = save_dir / f"{regime_name}_model.keras"
            metadata_path = save_dir / f"{regime_name}_metadata.json"

            if not model_path.exists():
                raise FileNotFoundError(f"No saved model found for regime {regime_name}")

            # Load model
            model = tf.keras.models.load_model(model_path)

            # Load metadata
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)

            # Restore scaler parameters
            self.scaler.scale_ = np.array(metadata['scaler_params']['scale_'])
            self.scaler.min_ = np.array(metadata['scaler_params']['min_'])

            self.models[regime_name] = model
            self.regime_metadata[regime_name] = metadata

            return model, metadata

        except Exception as e:
            print(f"Error loading regime {regime_name}: {str(e)}")
            raise

def main():
    try:
        print("Fetching SPY data...")
        df = yf.download('SPY', start='2014-01-01', end='2017-12-31')
        if df.empty:
            raise ValueError("No data downloaded")

        print(f"Downloaded {len(df)} days of data")

        # Create regime classifier
        regime_lstm = RegimeLSTM(lookback=50)

        # Define example regimes
        regimes = {
            "high_vol_uptrend": {
                "volatility": "high",
                "trend": "upward",
                "volume": "above_average"
            },
            "low_vol_sideways": {
                "volatility": "low",
                "trend": "sideways",
                "volume": "below_average"
            }
        }

        print("\nTraining regime models...")

        # Train models
        high_vol_period = df['Close'].values[:500]
        model_high_vol, metadata_high_vol = regime_lstm.train_regime(
            high_vol_period,
            "high_vol_uptrend",
            regimes["high_vol_uptrend"]
        )

        low_vol_period = df['Close'].values[500:1000]
        model_low_vol, metadata_low_vol = regime_lstm.train_regime(
            low_vol_period,
            "low_vol_sideways",
            regimes["low_vol_sideways"]
        )

        # Get exactly lookback days for prediction
        recent_prices = df['Close'].values[-regime_lstm.lookback:]
        print(f"\nMaking predictions using last {len(recent_prices)} price points")

        prediction_high_vol = regime_lstm.predict_with_regime(
            recent_prices,
            "high_vol_uptrend"
        )

        prediction_low_vol = regime_lstm.predict_with_regime(
            recent_prices,
            "low_vol_sideways"
        )

        # Print results
        print("\nRegime-Based LSTM Analysis")
        print("-" * 50)
        current_price = float(df['Close'].values[-1])
        print(f"Current Price: ${current_price:.2f}")
        print(f"High Volatility Regime Prediction: ${prediction_high_vol:.2f} ({((prediction_high_vol/current_price) - 1)*100:.2f}% change)")
        print(f"Low Volatility Regime Prediction: ${prediction_low_vol:.2f} ({((prediction_low_vol/current_price) - 1)*100:.2f}% change)")

        # Print regime characteristics
        print("\nStored Regime Characteristics:")
        for regime_name, metadata in regime_lstm.regime_metadata.items():
            print(f"\n{regime_name}:")
            for key, value in metadata['characteristics'].items():
                print(f"  {key}: {value}")

    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed

Fetching SPY data...
Downloaded 1007 days of data

Training regime models...

Training high_vol_uptrend model...
Training data shape: (450, 50, 1)
Epoch 1/50


12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - loss: 0.2181 - val_loss: 0.0701
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0240 - val_loss: 0.0184
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0131 - val_loss: 0.0150
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0057 - val_loss: 0.0139
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0050 - val_loss: 0.0134
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0046 - val_loss: 0.0139
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0041 - val_loss: 0.0137
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0045 - val_loss: 0.0136
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0042 - val_loss: 0.0137
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0044 - val_loss: 0.0138
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0043 - val_loss: 0.0134
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0042 - val_l

<ipython-input-11-b356cbf4c46b>:230: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from sklearn.model_selection import TimeSeriesSplit
import json
from pathlib import Path
import tensorflow as tf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class RegimeLSTM:
    def __init__(self, lookback=50):
        self.lookback = lookback
        self.scaler = MinMaxScaler()
        self.models = {}
        self.regime_metadata = {}
        self.training_history = {}

    def create_model(self):
        """Create model using functional API"""
        inputs = Input(shape=(self.lookback, 1))
        lstm_out = LSTM(50)(inputs)
        outputs = Dense(1)(lstm_out)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='mse')
        return model

    def prepare_data(self, prices, for_prediction=False):
        """Prepare data with validation and convert to tensor"""
        if for_prediction:
            if len(prices) != self.lookback:
                raise ValueError(f"For prediction, need exactly {self.lookback} price points, got {len(prices)}")
            scaled_data = self.scaler.fit_transform(prices.reshape(-1, 1))
            return tf.convert_to_tensor(scaled_data.reshape(1, self.lookback, 1), dtype=tf.float32), None
        else:
            if len(prices) < self.lookback + 1:
                raise ValueError(f"For training, need at least {self.lookback + 1} price points, got {len(prices)}")

            scaled_data = self.scaler.fit_transform(prices.reshape(-1, 1))
            X, y = [], []
            for i in range(self.lookback, len(scaled_data)):
                X.append(scaled_data[i-self.lookback:i])
                y.append(scaled_data[i])
            X_tensor = tf.convert_to_tensor(np.array(X), dtype=tf.float32)
            y_tensor = tf.convert_to_tensor(np.array(y), dtype=tf.float32)
            return X_tensor, y_tensor

    def train_regime(self, prices, regime_name, regime_characteristics):
        """Train and save model with error handling"""
        try:
            X, y = self.prepare_data(prices, for_prediction=False)

            batch_size = min(32, len(X))
            model = self.create_model()

            print(f"\nTraining {regime_name} model...")
            print(f"Training data shape: {X.shape}")

            history = model.fit(
                X, y,
                epochs=50,
                batch_size=batch_size,
                validation_split=0.2,
                verbose=1
            )

            # Store training history
            self.training_history[regime_name] = history

            save_dir = Path("regime_models")
            save_dir.mkdir(exist_ok=True)

            model_path = save_dir / f"{regime_name}_model.keras"
            metadata_path = save_dir / f"{regime_name}_metadata.json"

            model.save(model_path)

            metadata = {
                "regime_name": regime_name,
                "characteristics": regime_characteristics,
                "training_loss": float(history.history['loss'][-1]),
                "data_points": len(prices),
                "lookback": self.lookback,
                "scaler_params": {
                    "scale_": self.scaler.scale_.tolist(),
                    "min_": self.scaler.min_.tolist(),
                }
            }

            with open(metadata_path, 'w') as f:
                json.dump(metadata, f, indent=4)

            self.models[regime_name] = model
            self.regime_metadata[regime_name] = metadata

            return model, metadata

        except Exception as e:
            print(f"Error training regime {regime_name}: {str(e)}")
            raise

    def predict_with_regime(self, prices, regime_name):
        """Make predictions with error handling"""
        try:
            if regime_name not in self.models:
                self.load_regime_model(regime_name)

            if len(prices) != self.lookback:
                prices = prices[-self.lookback:]

            X, _ = self.prepare_data(prices, for_prediction=True)
            prediction = self.models[regime_name].predict(X, verbose=0)
            prediction_orig = self.scaler.inverse_transform(prediction)

            return float(prediction_orig[0][0])

        except Exception as e:
            print(f"Error predicting with regime {regime_name}: {str(e)}")
            raise

    def train_with_cross_validation(self, prices, regime_name, regime_characteristics, n_splits=5):
        """Train with time series cross-validation"""
        tscv = TimeSeriesSplit(n_splits=n_splits)
        cv_scores = []
        cv_predictions = []

        print(f"\nPerforming {n_splits}-fold time series cross-validation for {regime_name}")

        min_required = self.lookback * (n_splits + 1)
        if len(prices) < min_required:
            raise ValueError(f"Need at least {min_required} data points for {n_splits}-fold validation")

        for fold, (train_idx, val_idx) in enumerate(tscv.split(prices)):
            print(f"\nFold {fold + 1}/{n_splits}")

            train_prices = prices[train_idx]
            val_prices = prices[val_idx]

            if len(val_prices) <= self.lookback:
                print(f"Skipping fold {fold + 1} - insufficient validation data")
                continue

            X_train, y_train = self.prepare_data(train_prices, for_prediction=False)

            try:
                X_val, y_val = self.prepare_data(val_prices, for_prediction=False)
            except ValueError as e:
                print(f"Skipping fold {fold + 1}: {str(e)}")
                continue

            model = self.create_model()
            history = model.fit(
                X_train, y_train,
                epochs=50,
                batch_size=32,
                validation_data=(X_val, y_val),
                verbose=1
            )

            val_score = model.evaluate(X_val, y_val, verbose=0)
            cv_scores.append(val_score)

            val_pred = model.predict(X_val, verbose=0)
            cv_predictions.append({
                'true': self.scaler.inverse_transform(y_val.numpy()),
                'pred': self.scaler.inverse_transform(val_pred)
            })

        return cv_scores, cv_predictions

    def visualize_training(self, history, regime_name):
        """Create interactive training visualization"""
        fig = make_subplots(rows=2, cols=1,
                           subplot_titles=('Training & Validation Loss',
                                         'Learning Rate'))

        fig.add_trace(
            go.Scatter(y=history.history['loss'],
                      name='Training Loss',
                      line=dict(color='blue')),
            row=1, col=1
        )

        if 'val_loss' in history.history:
            fig.add_trace(
                go.Scatter(y=history.history['val_loss'],
                          name='Validation Loss',
                          line=dict(color='red')),
                row=1, col=1
            )

        if 'lr' in history.history:
            fig.add_trace(
                go.Scatter(y=history.history['lr'],
                          name='Learning Rate',
                          line=dict(color='green')),
                row=2, col=1
            )

        fig.update_layout(height=800,
                         title_text=f"Training Progress for {regime_name}",
                         showlegend=True)

        return fig

def enhanced_main():
    try:
        print("Fetching SPY data...")
        df = yf.download('SPY', start='2014-01-01', end='2017-12-31')

        regime_lstm = RegimeLSTM(lookback=50)

        regimes = {
            "high_vol_uptrend": {
                "volatility": "high",
                "trend": "upward",
                "volume": "above_average"
            },
            "low_vol_sideways": {
                "volatility": "low",
                "trend": "sideways",
                "volume": "below_average"
            }
        }

        predictions_dict = {}

        for regime_name, characteristics in regimes.items():
            print(f"\nProcessing {regime_name}...")
            train_data = df['Close'].values[:500]

            # Perform cross-validation
            cv_scores, cv_predictions = regime_lstm.train_with_cross_validation(
                train_data, regime_name, characteristics
            )

            print(f"\n{regime_name} Cross-Validation Results:")
            print(f"Mean MSE: {np.mean(cv_scores):.6f}")
            print(f"Std MSE: {np.std(cv_scores):.6f}")

            # Train final model
            model, metadata = regime_lstm.train_regime(
                train_data,
                regime_name,
                characteristics
            )

            # Visualize training
            training_viz = regime_lstm.visualize_training(
                regime_lstm.training_history[regime_name],
                regime_name
            )
            training_viz.show()

            # Make prediction
            recent_prices = df['Close'].values[-regime_lstm.lookback:]
            prediction = regime_lstm.predict_with_regime(
                recent_prices,
                regime_name
            )
            predictions_dict[regime_name] = prediction

        # Print results
        current_price = float(df['Close'].values[-1])
        print("\nPrediction Summary:")
        print("-" * 50)
        for regime_name, pred in predictions_dict.items():
            change_pct = ((pred/current_price) - 1) * 100
            print(f"{regime_name}:")
            print(f"  Prediction: ${pred:.2f}")
            print(f"  Expected Change: {change_pct:.2f}%")

    except Exception as e:
        print(f"Error in enhanced execution: {str(e)}")
        raise

if __name__ == "__main__":
    enhanced_main()

[*********************100%***********************]  1 of 1 completed

Fetching SPY data...

Processing high_vol_uptrend...

Performing 5-fold time series cross-validation for high_vol_uptrend

Fold 1/5
Epoch 1/50


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 293ms/step - loss: 0.7020 - val_loss: 0.4358
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.5486 - val_loss: 0.3366
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.4153 - val_loss: 0.2486
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.2966 - val_loss: 0.1693
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.1865 - val_loss: 0.1004
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0927 - val_loss: 0.0516
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.0276 - val_loss: 0.0475
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0236 - val_loss: 0.0789
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0606 - val_loss: 0.0729
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0540 - val_loss: 0.0528
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0276 - val_loss: 0.0413
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0158 - val_loss: 0.0397
Epoch 13/50


Processing low_vol_sideways...

Performing 5-fold time series cross-validation for low_vol_sideways

Fold 1/5
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 321ms/step - loss: 0.8446 - val_loss: 0.5072
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.6461 - val_loss: 0.3771
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.4666 - val_loss: 0.2658
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.3197 - val_loss: 0.1714
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.1917 - val_loss: 0.0963
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 0.0898 - val_loss: 0.0489
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0258 - val_loss: 0.0464
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0225 - val_loss: 0.0802
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.0635 - val_loss: 0.0850
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0692 - val_loss: 0.0655
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss:


Prediction Summary:
--------------------------------------------------
high_vol_uptrend:
  Prediction: $267.50
  Expected Change: 0.24%
low_vol_sideways:
  Prediction: $266.65
  Expected Change: -0.08%


<ipython-input-14-1e23500e2626>:267: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [20]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from sklearn.model_selection import TimeSeriesSplit
import json
from pathlib import Path
import tensorflow as tf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class RegimeLSTM:
    def __init__(self, lookback=50):
        self.lookback = lookback
        self.scaler = MinMaxScaler()
        self.models = {}
        self.regime_metadata = {}
        self.training_history = {}

    def create_model(self):
        """Create model using functional API"""
        inputs = Input(shape=(self.lookback, 1))
        lstm_out = LSTM(50)(inputs)
        outputs = Dense(1)(lstm_out)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='mse')
        return model

    def prepare_data(self, prices, for_prediction=False):
        """Prepare data with validation and convert to tensor"""
        if for_prediction:
            if len(prices) != self.lookback:
                raise ValueError(f"For prediction, need exactly {self.lookback} price points, got {len(prices)}")
            scaled_data = self.scaler.fit_transform(prices.reshape(-1, 1))
            return tf.convert_to_tensor(scaled_data.reshape(1, self.lookback, 1), dtype=tf.float32), None
        else:
            if len(prices) < self.lookback + 1:
                raise ValueError(f"For training, need at least {self.lookback + 1} price points, got {len(prices)}")

            scaled_data = self.scaler.fit_transform(prices.reshape(-1, 1))
            X, y = [], []
            for i in range(self.lookback, len(scaled_data)):
                X.append(scaled_data[i-self.lookback:i])
                y.append(scaled_data[i])
            X_tensor = tf.convert_to_tensor(np.array(X), dtype=tf.float32)
            y_tensor = tf.convert_to_tensor(np.array(y), dtype=tf.float32)
            return X_tensor, y_tensor

    def train_regime(self, prices, regime_name, regime_characteristics):
        """Train regime with progress tracking"""
        try:
            print(f"\nPreparing data for {regime_name}...")
            X, y = self.prepare_data(prices, for_prediction=False)

            batch_size = min(32, len(X))
            print(f"Using batch size: {batch_size}")

            print(f"Creating model for {regime_name}...")
            model = self.create_model()

            print(f"\nTraining {regime_name} model...")
            print(f"Training data shape: {X.shape}")

            history = model.fit(
                X, y,
                epochs=25,
                batch_size=batch_size,
                validation_split=0.2,
                verbose=1
            )

            self.training_history[regime_name] = history
            self.models[regime_name] = model

            # Save metadata
            metadata = {
                "regime_name": regime_name,
                "characteristics": regime_characteristics,
                "training_loss": float(history.history['loss'][-1]),
                "data_points": len(prices),
                "lookback": self.lookback
            }
            self.regime_metadata[regime_name] = metadata

            print(f"Completed training for {regime_name}")
            return model, metadata

        except Exception as e:
            print(f"Error training regime {regime_name}: {str(e)}")
            raise

    def predict_with_regime(self, prices, regime_name):
        """Make predictions with error handling"""
        try:
            if len(prices) != self.lookback:
                prices = prices[-self.lookback:]

            X, _ = self.prepare_data(prices, for_prediction=True)
            prediction = self.models[regime_name].predict(X, verbose=0)
            prediction_orig = self.scaler.inverse_transform(prediction)

            return float(prediction_orig[0][0])

        except Exception as e:
            print(f"Error predicting with regime {regime_name}: {str(e)}")
            raise

    def train_with_cross_validation(self, prices, regime_name, regime_characteristics, n_splits=3):
        """Train with cross-validation and progress tracking"""
        tscv = TimeSeriesSplit(n_splits=n_splits)
        cv_scores = []
        cv_predictions = []

        print(f"\nPerforming {n_splits}-fold time series cross-validation for {regime_name}")

        for fold, (train_idx, val_idx) in enumerate(tscv.split(prices)):
            print(f"\nStarting fold {fold + 1}/{n_splits}")

            train_prices = prices[train_idx]
            val_prices = prices[val_idx]

            print(f"Training data size: {len(train_prices)}")
            print(f"Validation data size: {len(val_prices)}")

            try:
                X_train, y_train = self.prepare_data(train_prices, for_prediction=False)
                X_val, y_val = self.prepare_data(val_prices, for_prediction=False)

                model = self.create_model()

                print(f"Training fold {fold + 1}...")
                history = model.fit(
                    X_train, y_train,
                    epochs=25,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    verbose=1
                )

                val_score = model.evaluate(X_val, y_val, verbose=0)
                cv_scores.append(val_score)

                print(f"Fold {fold + 1} validation score: {val_score:.6f}")

            except Exception as e:
                print(f"Error in fold {fold + 1}: {str(e)}")
                continue

        return cv_scores

    def visualize_training(self, regime_name):
        """Visualize training progress"""
        if regime_name not in self.training_history:
            print(f"No training history found for {regime_name}")
            return None

        history = self.training_history[regime_name]
        fig = go.Figure()

        # Plot training loss
        fig.add_trace(
            go.Scatter(
                y=history.history['loss'],
                name='Training Loss',
                line=dict(color='blue')
            )
        )

        # Plot validation loss if available
        if 'val_loss' in history.history:
            fig.add_trace(
                go.Scatter(
                    y=history.history['val_loss'],
                    name='Validation Loss',
                    line=dict(color='red')
                )
            )

        fig.update_layout(
            title=f"Training Progress for {regime_name}",
            xaxis_title="Epoch",
            yaxis_title="Loss",
            showlegend=True
        )

        return fig

def main():
    try:
        print("Fetching SPY data...")
        df = yf.download('SPY', start='2014-01-01', end='2017-12-31')
        print(f"Downloaded {len(df)} days of data")

        regime_lstm = RegimeLSTM(lookback=50)

        regimes = {
            "high_vol_uptrend": {
                "volatility": "high",
                "trend": "upward",
                "volume": "above_average"
            },
            "low_vol_sideways": {
                "volatility": "low",
                "trend": "sideways",
                "volume": "below_average"
            }
        }

        predictions = {}

        for regime_name, characteristics in regimes.items():
            print(f"\nProcessing {regime_name}...")
            train_data = df['Close'].values[:500]

            # Perform cross-validation
            cv_scores = regime_lstm.train_with_cross_validation(
                train_data,
                regime_name,
                characteristics
            )

            print(f"\n{regime_name} Cross-Validation Results:")
            print(f"Mean MSE: {np.mean(cv_scores):.6f}")
            print(f"Std MSE: {np.std(cv_scores):.6f}")

            # Train final model
            model, metadata = regime_lstm.train_regime(
                train_data,
                regime_name,
                characteristics
            )

            # Visualize training
            fig = regime_lstm.visualize_training(regime_name)
            if fig is not None:
                fig.show()

            # Make prediction
            recent_prices = df['Close'].values[-regime_lstm.lookback:]
            prediction = regime_lstm.predict_with_regime(
                recent_prices,
                regime_name
            )
            predictions[regime_name] = prediction

        # Print results
        print("\nPrediction Results:")
        print("-" * 50)
        current_price = float(df['Close'].values[-1])

        for regime_name, pred in predictions.items():
            change_pct = ((pred/current_price) - 1) * 100
            print(f"\n{regime_name}:")
            print(f"Current Price: ${current_price:.2f}")
            print(f"Predicted Price: ${pred:.2f}")
            print(f"Predicted Change: {change_pct:.2f}%")

    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed

Fetching SPY data...
Downloaded 1007 days of data

Processing high_vol_uptrend...

Performing 3-fold time series cross-validation for high_vol_uptrend

Starting fold 1/3
Training data size: 125
Validation data size: 125
Training fold 1...
Epoch 1/25


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 181ms/step - loss: 0.4055 - val_loss: 0.2820
Epoch 2/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.2668 - val_loss: 0.1643
Epoch 3/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.1343 - val_loss: 0.0712
Epoch 4/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0393 - val_loss: 0.0375
Epoch 5/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0121 - val_loss: 0.0786
Epoch 6/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0390 - val_loss: 0.0568
Epoch 7/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0185 - val_loss: 0.0350
Epoch 8/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0070 - val_loss: 0.0321
Epoch 9/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0094 - val_loss: 0.0337
Epoch 10/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0130 - val_loss: 0.0338
Epoch 11/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - loss: 0.0133 - val_loss: 0.0319
Epoch 12/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 0.0106 - val_loss: 0.0303
Epoch 13/25


Processing low_vol_sideways...

Performing 3-fold time series cross-validation for low_vol_sideways

Starting fold 1/3
Training data size: 125
Validation data size: 125
Training fold 1...
Epoch 1/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 166ms/step - loss: 0.5050 - val_loss: 0.3531
Epoch 2/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.3394 - val_loss: 0.2026
Epoch 3/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.1707 - val_loss: 0.0896
Epoch 4/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0548 - val_loss: 0.0399
Epoch 5/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 0.0114 - val_loss: 0.0857
Epoch 6/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0428 - val_loss: 0.0756
Epoch 7/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0306 - val_loss: 0.0427
Epoch 8/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0075 - val_loss: 0.0349
Epoch 9/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0087 - val_loss: 0.0363
Epoch 10/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.013


Prediction Results:
--------------------------------------------------

high_vol_uptrend:
Current Price: $266.86
Predicted Price: $266.75
Predicted Change: -0.04%

low_vol_sideways:
Current Price: $266.86
Predicted Price: $267.08
Predicted Change: 0.08%


<ipython-input-20-39edfa5f859f>:251: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [24]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from sklearn.model_selection import TimeSeriesSplit
import json
from pathlib import Path
import tensorflow as tf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class RegimeLSTM:
    def __init__(self, lookback=50):
        self.lookback = lookback
        self.scaler = MinMaxScaler()
        self.models = {}
        self.regime_metadata = {}
        self.training_history = {}

    def create_model(self):
        inputs = Input(shape=(self.lookback, 1))
        lstm_out = LSTM(50)(inputs)
        outputs = Dense(1)(lstm_out)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='mse')
        return model

    def prepare_data(self, prices, for_prediction=False):
        if for_prediction:
            if len(prices) != self.lookback:
                raise ValueError(f"For prediction, need exactly {self.lookback} price points, got {len(prices)}")
            scaled_data = self.scaler.fit_transform(prices.reshape(-1, 1))
            return tf.convert_to_tensor(scaled_data.reshape(1, self.lookback, 1), dtype=tf.float32), None
        else:
            if len(prices) < self.lookback + 1:
                raise ValueError(f"For training, need at least {self.lookback + 1} price points, got {len(prices)}")

            scaled_data = self.scaler.fit_transform(prices.reshape(-1, 1))
            X, y = [], []
            for i in range(self.lookback, len(scaled_data)):
                X.append(scaled_data[i-self.lookback:i])
                y.append(scaled_data[i])
            X_tensor = tf.convert_to_tensor(np.array(X), dtype=tf.float32)
            y_tensor = tf.convert_to_tensor(np.array(y), dtype=tf.float32)
            return X_tensor, y_tensor

    def train_regime(self, prices, regime_name, regime_characteristics):
        try:
            print(f"\nPreparing data for {regime_name}...")
            X, y = self.prepare_data(prices, for_prediction=False)

            batch_size = min(32, len(X))
            print(f"Using batch size: {batch_size}")

            model = self.create_model()
            print(f"\nTraining {regime_name} model...")
            print(f"Training data shape: {X.shape}")

            history = model.fit(
                X, y,
                epochs=25,
                batch_size=batch_size,
                validation_split=0.2,
                verbose=1
            )

            self.training_history[regime_name] = history
            self.models[regime_name] = model

            metadata = {
                "regime_name": regime_name,
                "characteristics": regime_characteristics,
                "training_loss": float(history.history['loss'][-1]),
                "data_points": len(prices),
                "lookback": self.lookback
            }
            self.regime_metadata[regime_name] = metadata

            print(f"Completed training for {regime_name}")
            return model, metadata

        except Exception as e:
            print(f"Error training regime {regime_name}: {str(e)}")
            raise

    def predict_with_regime(self, prices, regime_name):
        try:
            if len(prices) != self.lookback:
                prices = prices[-self.lookback:]

            X, _ = self.prepare_data(prices, for_prediction=True)
            prediction = self.models[regime_name].predict(X, verbose=0)
            prediction_orig = self.scaler.inverse_transform(prediction)

            return float(prediction_orig[0][0])

        except Exception as e:
            print(f"Error predicting with regime {regime_name}: {str(e)}")
            raise

    def visualize_ohlc(self, df):
        fig = make_subplots(rows=2, cols=1,
                           shared_xaxes=True,
                           vertical_spacing=0.05,
                           row_heights=[0.7, 0.3])

        fig.add_trace(
            go.Candlestick(
                x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                name='OHLC'
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Bar(
                x=df.index,
                y=df['Volume'],
                name='Volume',
                marker=dict(
                    color='rgba(100, 100, 100, 0.3)'
                )
            ),
            row=2, col=1
        )

        fig.update_layout(
            title='SPY Price Action',
            yaxis_title='Price',
            yaxis2_title='Volume',
            xaxis_rangeslider_visible=False
        )

        return fig

    def analyze_feature_importance(self, prices, regime_name):
        base_prediction = self.predict_with_regime(prices, regime_name)
        importance_scores = []

        for i in range(self.lookback):
            perturbed_prices = prices.copy()
            perturbed_prices[i] *= 1.01

            perturbed_prediction = self.predict_with_regime(perturbed_prices, regime_name)
            importance = abs((perturbed_prediction - base_prediction) / base_prediction * 100)
            importance_scores.append(importance)

        return importance_scores

    def visualize_feature_importance(self, importance_scores, regime_name):
        fig = go.Figure()

        fig.add_trace(
            go.Bar(
                x=list(range(self.lookback)),
                y=importance_scores,
                name='Feature Importance',
                marker=dict(
                    color=importance_scores,
                    colorscale='Viridis'
                )
            )
        )

        fig.update_layout(
            title=f'Feature Importance Analysis for {regime_name}',
            xaxis_title='Days Back',
            yaxis_title='Price Sensitivity (%)',
            showlegend=False
        )

        return fig

def main():
    try:
        print("Fetching SPY data...")
        df = yf.download('SPY', start='2014-01-01', end='2017-12-31')
        df = df[df.index.dayofweek < 5]  # Remove weekends
        print(f"Downloaded {len(df)} trading days")

        regime_lstm = RegimeLSTM(lookback=50)

        # Create and show OHLC chart
        ohlc_fig = regime_lstm.visualize_ohlc(df)
        ohlc_fig.show()

        regimes = {
            "high_vol_uptrend": {
                "volatility": "high",
                "trend": "upward",
                "volume": "above_average"
            },
            "low_vol_sideways": {
                "volatility": "low",
                "trend": "sideways",
                "volume": "below_average"
            }
        }

        predictions = {}

        for regime_name, characteristics in regimes.items():
            print(f"\nProcessing {regime_name}...")
            train_data = df['Close'].values[:500]

            # Train model
            model, metadata = regime_lstm.train_regime(
                train_data,
                regime_name,
                characteristics
            )

            # Make prediction
            recent_prices = df['Close'].values[-regime_lstm.lookback:]
            prediction = regime_lstm.predict_with_regime(
                recent_prices,
                regime_name
            )
            predictions[regime_name] = prediction

            # Analyze and visualize feature importance
            importance_scores = regime_lstm.analyze_feature_importance(
                recent_prices,
                regime_name
            )

            importance_fig = regime_lstm.visualize_feature_importance(
                importance_scores,
                regime_name
            )
            importance_fig.show()

        # Print results
        print("\nPrediction Results:")
        print("-" * 50)
        current_price = float(df['Close'].values[-1])

        for regime_name, pred in predictions.items():
            change_pct = ((pred/current_price) - 1) * 100
            print(f"\n{regime_name}:")
            print(f"Current Price: ${current_price:.2f}")
            print(f"Predicted Price: ${pred:.2f}")
            print(f"Predicted Change: {change_pct:.2f}%")

    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()

[*********************100%***********************]  1 of 1 completed

Fetching SPY data...
Downloaded 1007 trading days



Processing high_vol_uptrend...

Preparing data for high_vol_uptrend...
Using batch size: 32

Training high_vol_uptrend model...
Training data shape: (450, 50, 1)
Epoch 1/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 0.2456 - val_loss: 0.0537
Epoch 2/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0287 - val_loss: 0.0153
Epoch 3/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0101 - val_loss: 0.0132
Epoch 4/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0058 - val_loss: 0.0150
Epoch 5/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0044 - val_loss: 0.0130
Epoch 6/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0053 - val_loss: 0.0139
Epoch 7/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0047 - val_loss: 0.0134
Epoch 8/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0044 - val_loss: 0.0135
Epoch 9/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.0042 - val_loss: 0.0133
Epoch 10/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0040 - val


Processing low_vol_sideways...

Preparing data for low_vol_sideways...
Using batch size: 32

Training low_vol_sideways model...
Training data shape: (450, 50, 1)
Epoch 1/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - loss: 0.4017 - val_loss: 0.0402
Epoch 2/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0336 - val_loss: 0.0207
Epoch 3/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0152 - val_loss: 0.0186
Epoch 4/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - loss: 0.0086 - val_loss: 0.0225
Epoch 5/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0084 - val_loss: 0.0180
Epoch 6/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0070 - val_loss: 0.0198
Epoch 7/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0066 - val_loss: 0.0181
Epoch 8/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0059 - val_loss: 0.0191
Epoch 9/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0064 - val_loss: 0.0182
Epoch 10/25
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0062 - val


Prediction Results:
--------------------------------------------------

high_vol_uptrend:
Current Price: $266.86
Predicted Price: $266.89
Predicted Change: 0.01%

low_vol_sideways:
Current Price: $266.86
Predicted Price: $266.74
Predicted Change: -0.04%


<ipython-input-24-852cb51f7e75>:242: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

